## Cwiczenie 6
Napisz program, który wrzuci do bazy dokument, pobierze go i wypisze, zmodyfikuje go, 
następnie pobierze i wypisze, a na końcu usunie go i spróbuje pobrać z bazy.

In [56]:
import requests
import json

### Program pobiera aktalne kursy walut za pomoc API Narodowego Banku Polskiego, wybiera potrzebne waluty i zapisuje do bazy danych Riak

In [57]:
nbp_url = 'http://api.nbp.pl/api/exchangerates/tables/A/'
response = requests.get(nbp_url)
response.content.decode()

'[{"table":"A","no":"121/A/NBP/2022","effectiveDate":"2022-06-24","rates":[{"currency":"bat (Tajlandia)","code":"THB","mid":0.1258},{"currency":"dolar amerykański","code":"USD","mid":4.4656},{"currency":"dolar australijski","code":"AUD","mid":3.0842},{"currency":"dolar Hongkongu","code":"HKD","mid":0.5690},{"currency":"dolar kanadyjski","code":"CAD","mid":3.4439},{"currency":"dolar nowozelandzki","code":"NZD","mid":2.8133},{"currency":"dolar singapurski","code":"SGD","mid":3.2175},{"currency":"euro","code":"EUR","mid":4.7094},{"currency":"forint (Węgry)","code":"HUF","mid":0.011749},{"currency":"frank szwajcarski","code":"CHF","mid":4.6538},{"currency":"funt szterling","code":"GBP","mid":5.4831},{"currency":"hrywna (Ukraina)","code":"UAH","mid":0.1467},{"currency":"jen (Japonia)","code":"JPY","mid":0.033103},{"currency":"korona czeska","code":"CZK","mid":0.1903},{"currency":"korona duńska","code":"DKK","mid":0.6330},{"currency":"korona islandzka","code":"ISK","mid":0.033711},{"currency

In [58]:
date = response.json()[0]['effectiveDate']
rates = response.json()[0]['rates']
riak_currencies = ['EUR', 'USD', 'GBP']
riak_rates = {}
for cur in rates:
    if cur['code'] in riak_currencies:
        riak_rates[cur['code']] = cur['mid']
        
riak_url = 'http://3.88.140.204:8098/riak/rates/'+ date

In [59]:
headers = {'Content-Type': 'application/json'}
response = requests.put(riak_url, headers=headers, data=json.dumps(riak_rates))
print(response.status_code)
print(response.headers)

204
{'Vary': 'Accept-Encoding', 'Server': 'MochiWeb/1.1 WebMachine/1.10.6 (no drinks)', 'Date': 'Sun, 26 Jun 2022 13:45:43 GMT', 'Content-Type': 'application/json', 'Content-Length': '0', 'Content-Encoding': 'gzip'}


In [60]:
response = requests.get(riak_url)
print(response.status_code)
print(response.headers)
print(response.json())

200
{'X-Riak-Vclock': 'a85hYGBgzGDKBVIcBXm1//YGXViTwZTIlMfKwH5f5w5fFgA=', 'Vary': 'Accept-Encoding', 'Server': 'MochiWeb/1.1 WebMachine/1.10.6 (no drinks)', 'Link': '</riak/rates>; rel="up"', 'Last-Modified': 'Sun, 26 Jun 2022 13:45:43 GMT', 'ETag': '"74FNIJKZ2FmHMXWbJE1N7X"', 'Date': 'Sun, 26 Jun 2022 13:45:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '59', 'Content-Encoding': 'gzip'}
{'USD': 4.4656, 'EUR': 4.7094, 'GBP': 5.4831}


In [61]:
response = requests.delete(riak_url)
print(response.status_code)
print(response.headers)

204
{'Vary': 'Accept-Encoding', 'Server': 'MochiWeb/1.1 WebMachine/1.10.6 (no drinks)', 'Date': 'Sun, 26 Jun 2022 13:45:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '0', 'Content-Encoding': 'gzip'}
